In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# エージェントの基本クラス
class Agent:
    def __init__(self, initial_action):
        self.action = initial_action
        self.total_payoff = 0.0

    def update_payoff(self, payoff):
        self.total_payoff += payoff

# ZD戦略を持つエージェントクラス
class ZDAgent(Agent):
    def __init__(self, x0ov, x0und, W, kappa, chi):
        self.actions = {"x_ov": x0ov, "x_und": x0und}
        initial_action_key = np.random.choice(["x_ov", "x_und"])
        super().__init__(self.actions[initial_action_key])
        self.W = W
        self.kappa = kappa
        self.chi = chi
        self.current_action_key = initial_action_key

    def choose_action(self, B):
        T0 = self.calculate_probability(B)
        if np.random.random() <= T0:
            self.action = self.actions["x_und"]
            self.current_action_key = "x_und"
        else:
            self.action = self.actions["x_ov"]
            self.current_action_key = "x_ov"

    def calculate_probability(self, B):
        return B / self.W + 1.0 if self.current_action_key == "x_und" else B / self.W

# 通常のエージェントクラス
class NormalAgent(Agent):
    def __init__(self, a, b, c, N):
        self.actions = {
            "xc": (a - c) / (2 * N * b),
            "xn": (a - c) / ((N + 1) * b)
        }
        initial_action_key = "xc" if np.random.random() < 0.5 else "xn"
        super().__init__(self.actions[initial_action_key])
        self.a = a
        self.b = b
        self.c = c
        self.N = N
        self.current_action_key = initial_action_key
        self.qarray = np.random.rand(4)

    def calc_payoff(self, qarray, agents, number, a, b, c):
        actions = []
        if qarray.all == self.qarray.all:
            for i, agent in enumerate(agents):
                actions.append(agent.action)
            xtot = sum(actions)
            # 利得を計算
            return ((a - b * xtot) * theta(a - b * xtot) - c) * actions[number]
        else:
            for i, agent in enumerate(agents):
                if i == number:
                    # 自分の仮定行動を `qarray` に基づいて計算
                    if np.random.random() < qarray[self.prev_action_index]:
                        actions.append(self.actions["xc"])
                    else:
                        actions.append(self.actions["xn"])
                else:
                    actions.append(agent.action)
            xtot = sum(actions)
            # 利得を計算
            return ((a - b * xtot) * theta(a - b * xtot) - c) * actions[number], actions[number]

    def choose_action(self, zd_action, other_actions, zd_actions, agents, number, a, b, c, t):
        zd_discrete = 0 if zd_action == zd_actions["x_und"] else 1
        self_discrete = 0 if self.action == self.actions["xc"] else 1
        other_count = sum(1 for action in other_actions if action == self.actions["xc"])
        self.prev_action_index = zd_discrete  + self_discrete * 2
        #print(self.prev_action_index)

        if t != 0:
            if np.random.random() < self.qarray[self.prev_action_index]:
                self.action = self.actions["xc"]
                self.current_action_key = "xc"
            else:
                self.action = self.actions["xn"]
                self.current_action_key = "xn"

        """ランダムにqarrayを変化させて期待利得を比較し、改善があれば採用する。"""
        # qarrayの変更を作成 (+0.001 または -0.001)
        delta = np.zeros_like(self.qarray)  # deltaの初期化
        delta[self.prev_action_index] = np.random.choice([-0.001, 0.001])  # 指定されたインデックスだけ変更
        new_qarray = np.clip(self.qarray + delta, 0.0, 1.0)  # [0, 1]の範囲に制限

        # 期待利得を計算
        current_payoff = self.calc_payoff(self.qarray, agents, number, a, b, c)
        new_payoff, new_action = self.calc_payoff(new_qarray, agents, number, a, b, c)

        # 改善があれば新しいqarrayを採用
        if new_payoff > current_payoff:
            self.qarray = new_qarray
            self.action = new_action
            return True
        return False
        
# ステップ関数
def theta(y):
    return 1.0 if y >= 0 else 0.0

# 各エージェントの利得を計算
def calculate_payoffs(agents, a, b, c):
    xtot = sum(agent.action for agent in agents)
    for agent in agents:
        payoff = ((a - b * xtot) * theta(a - b * xtot) - c) * agent.action
        agent.update_payoff(payoff)

# シミュレーション実行
def run_simulation(N, NTIME, a, b, c, xmax, chi, kappa, trial):
    results = []
    prob_history = []
    payoff_history = []
    W = 2.0 * c * xmax + abs((chi - 1) * kappa)
    x0ov = chi / (chi + N - 1.0) * (a - c) / b
    x0und = xmax
    zd_actions = {"x_ov": x0ov, "x_und": x0und}
    # 各アクションのカウントを初期化
    action_counts = {"x_ov": 0, "x_und": 0, "xc": 0, "xn": 0}
    agents = [ZDAgent(x0ov, x0und, W, kappa, chi)]
    agents.extend(NormalAgent(a, b, c, N) for _ in range(1, N))

    B = agents[0].total_payoff + (chi - 1.0) * kappa
    for agent in agents[1:]:
        B -= chi / (N - 1) * agent.total_payoff

    s = 0
    k = 0
    # データ保存用
    with open(f"simulation_results_all_{trial}.txt", "w") as file:
        file.write("TimeStep, S0, Smj, Actions, qarray, improved, improved step\n")  # ヘッダー行
        for t in tqdm(range(NTIME), desc="Simulation Progress"):
            previous_actions = [agent.action for agent in agents]
            improved = False

            for i in range(1, N):
                other_actions = [previous_actions[j] for j in range(len(agents)) if j != i]
                if agents[i].choose_action(agents[0].action, other_actions, zd_actions, agents, i, a, b, c, t):
                    improved = True
                    k = t

                qarray_str = ",".join(map(str, agents[i].qarray))

            agents[0].choose_action(B)
            calculate_payoffs(agents, a, b, c)
            B = (
                agents[0].total_payoff
                - chi / (N - 1) * sum(agent.total_payoff for agent in agents[1:])
                + (chi - 1.0) * kappa
            )
            # アクションをカウント
            if agents[0].current_action_key == "x_ov":
                action_counts["x_ov"] += 1
            elif agents[0].current_action_key == "x_und":
                action_counts["x_und"] += 1

            for agent in agents[1:]:
                if agent.current_action_key == "xc":
                    action_counts["xc"] += 1
                elif agent.current_action_key == "xn":
                    action_counts["xn"] += 1

            S0 = agents[0].action * ((a - b * sum(agent.action for agent in agents)) * theta(a - b * sum(agent.action for agent in agents)) - c)
            Smj = np.mean([
                agent.action * ((a - b * sum(agent.action for agent in agents)) * theta(a - b * sum(agent.action for agent in agents)) - c)
                for agent in agents[1:]
            ])
            results.append((S0, Smj))

            current_payoffs = [agent.action * ((a - b * sum(agent.action for agent in agents)) * theta(a - b * sum(agent.action for agent in agents)) - c) for agent in agents]
            payoff_history.append(current_payoffs)  # 各時点のエージェントの利得を追加
            actions_str = ",".join(map(str, [agent.action for agent in agents]))
            file.write(f"{t},{S0},{Smj},{actions_str},{qarray_str}, {improved}, {k}\n")
            
            if improved:
                s += 1
                prob_history.append(np.mean([agent.qarray for agent in agents[1:]], axis=0))
                
    # 最後に NumPy 配列へ変換（NTIME × N の形になる）
    payoff_history = np.array(payoff_history)

    # payoff_history（生データ）をファイル保存
    np.savetxt(f"payoff_history_all_{trial}.txt", payoff_history, fmt="%.6f", delimiter=",")

    """
    # アクションカウントをメモに保存
    with open("action_counts_summary6.txt", "w") as memo_file:
        memo_file.write("Action Counts Summary:\n")
        memo_file.write(f"x_ov: {action_counts['x_ov']}\n")
        memo_file.write(f"x_und: {action_counts['x_und']}\n")
        memo_file.write(f"xc: {action_counts['xc']}\n")
        memo_file.write(f"xn: {action_counts['xn']}\n")
    """

    return results, np.array(prob_history), payoff_history, t+1

if __name__ == "__main__":
    N = 2  # エージェント数
    NTIME = 100000000 # タイムステップ数
    a, b, c, xmax = 2.0, 1.0, 1.0, 2.5
    chi, kappa = 1.0, 0.0

    for trial in range(1, 101):  # 100回シミュレーション
        print(f"=== Trial {trial} / 100 ===")
        results, prob_history, payoff_history, s = run_simulation(N, NTIME, a, b, c, xmax, chi, kappa, trial)


=== Trial 58 / 100 ===


Simulation Progress: 100%|██████████| 100000000/100000000 [2:30:21<00:00, 11084.36it/s] 
